# Lab 05 · Release Readiness Agent and Tools

*This lab notebook provides guided steps. All commands are intended for local execution.*

## Objectives
- Model a deterministic release readiness agent that can plug into the existing AI web app.
- Provide credible tool abstractions for product briefs, launch windows, and stakeholder contacts.
- Expose an `/ai/release-readiness` endpoint that returns structured recommendations and planner output.

## What will be learned
- Coordinating multiple service helpers inside an agent workflow.
- Returning Pydantic models that the frontend can render without extra parsing.
- Logging tool invocations to aid observability and debugging.

## Prerequisites & install
Reuse the virtual environment created earlier in the course. No additional dependencies are required beyond `pydantic`.

```bash
cd ai-web/backend
. .venv/bin/activate
pip install pydantic
```

## Step-by-step tasks
Build out the tools, service, and router layers needed to run the release readiness agent.

### Step 1: Release data tools
Capture deterministic product data in `app/services/agent_tools.py` so the agent can make realistic decisions.

In [ ]:
from pathlib import Path

tools_path = Path('ai-web/backend/app/services/agent_tools.py')
tools_path.parent.mkdir(parents=True, exist_ok=True)
tools_path.write_text('"""Deterministic tool helpers used by the release readiness agent."""\n\nfrom __future__ import annotations\n\nfrom datetime import date\nfrom typing import Literal\n\nfrom pydantic import BaseModel, Field\n\n\nclass FeatureBrief(BaseModel):\n    """Condensed product brief for a feature under development."""\n\n    slug: str\n    name: str\n    summary: str\n    audience_role: str\n    audience_experience: Literal["beginner", "intermediate", "advanced"]\n    success_metric: str\n\n\nclass LaunchWindow(BaseModel):\n    """Deployment window information tracked by the release team."""\n\n    feature_slug: str\n    environment: Literal["staging", "production"]\n    window_start: date\n    window_end: date\n    freeze_required: bool = Field(default=True)\n    notes: str = Field(default="")\n\n\nclass SupportContact(BaseModel):\n    """Contact details for teams who need proactive updates."""\n\n    audience: str\n    contact: str\n    escalation_channel: str\n\n\n_FEATURE_BRIEFS: dict[str, FeatureBrief] = {\n    "curriculum-pathways": FeatureBrief(\n        slug="curriculum-pathways",\n        name="Curriculum Pathways",\n        summary=(\n            "Surface sequenced lab recommendations so instructors can scaffold lessons "\n            "for students based on prior completions."\n        ),\n        audience_role="Instructor",\n        audience_experience="intermediate",\n        success_metric="90% of instructors adopt generated pathways for the next cohort",\n    ),\n    "team-analytics": FeatureBrief(\n        slug="team-analytics",\n        name="Team Analytics Dashboard",\n        summary="Deliver a consolidated dashboard that highlights agent usage and completion trends for admins.",\n        audience_role="Program Manager",\n        audience_experience="advanced",\n        success_metric="Daily active program managers increase by 25%",\n    ),\n}\n\n_LAUNCH_WINDOWS: dict[str, LaunchWindow] = {\n    "curriculum-pathways": LaunchWindow(\n        feature_slug="curriculum-pathways",\n        environment="production",\n        window_start=date(2025, 3, 10),\n        window_end=date(2025, 3, 12),\n        freeze_required=True,\n        notes="Coordinated release with marketing webinar on Mar 11.",\n    ),\n    "team-analytics": LaunchWindow(\n        feature_slug="team-analytics",\n        environment="production",\n        window_start=date(2025, 4, 2),\n        window_end=date(2025, 4, 4),\n        freeze_required=True,\n        notes="Requires feature flag rollout 48 hours prior to launch.",\n    ),\n}\n\n_SUPPORT_DIRECTORY: dict[str, list[SupportContact]] = {\n    "Instructor": [\n        SupportContact(\n            audience="Instructor",\n            contact="education-success@example.com",\n            escalation_channel="#instructor-support",\n        ),\n        SupportContact(\n            audience="Instructor",\n            contact="pedagogy-lead@example.com",\n            escalation_channel="#curriculum-updates",\n        ),\n    ],\n    "Program Manager": [\n        SupportContact(\n            audience="Program Manager",\n            contact="program-ops@example.com",\n            escalation_channel="#program-ops",\n        ),\n    ],\n}\n\n_SLO_WATCH_ITEMS: dict[str, list[str]] = {\n    "curriculum-pathways": [\n        "Lesson ingestion latency must stay under 2 minutes",\n        "Planner responses require >95% schema compliance",\n    ],\n    "team-analytics": [\n        "Dashboard queries should resolve under 1.5 seconds",\n        "Background aggregation jobs must remain below 75% CPU utilization",\n    ],\n}\n\n\ndef fetch_feature_brief(feature_slug: str) -> FeatureBrief:\n    """Return the canonical product brief for the requested feature."""\n\n    brief = _FEATURE_BRIEFS.get(feature_slug)\n    if brief is None:\n        raise KeyError(feature_slug)\n    return brief\n\n\ndef fetch_launch_window(feature_slug: str) -> LaunchWindow:\n    """Fetch the release window associated with the feature."""\n\n    window = _LAUNCH_WINDOWS.get(feature_slug)\n    if window is None:\n        raise KeyError(feature_slug)\n    return window\n\n\ndef fetch_support_contacts(audience_role: str) -> list[SupportContact]:\n    """Return the set of contacts who should be looped in for updates."""\n\n    contacts = _SUPPORT_DIRECTORY.get(audience_role)\n    if contacts:\n        return contacts\n    return [\n        SupportContact(\n            audience=audience_role,\n            contact="success@example.com",\n            escalation_channel="#general-updates",\n        )\n    ]\n\n\ndef list_slo_watch_items(feature_slug: str) -> list[str]:\n    """List performance and reliability signals for the feature."""\n\n    return _SLO_WATCH_ITEMS.get(feature_slug, [])\n')
print('Agent tool helpers ready.')

### Step 2: Release readiness agent service
Implement `app/services/agent.py` to orchestrate tool calls and produce structured recommendations.

In [ ]:
from pathlib import Path

agent_service_path = Path('ai-web/backend/app/services/agent.py')
agent_service_path.parent.mkdir(parents=True, exist_ok=True)
agent_service_path.write_text('"""Release readiness agent service orchestrating deterministic tool calls."""\n\nfrom __future__ import annotations\n\nfrom datetime import date\nfrom typing import Any, Literal\n\nfrom pydantic import BaseModel, Field\n\nfrom app.schemas.planner import Plan, PlanRequest\nfrom app.services.agent_tools import (\n    FeatureBrief,\n    LaunchWindow,\n    SupportContact,\n    fetch_feature_brief,\n    fetch_launch_window,\n    fetch_support_contacts,\n    list_slo_watch_items,\n)\nfrom app.services.planner import build_plan\n\n\nclass AgentServiceError(RuntimeError):\n    """Raised when the agent cannot complete its workflow."""\n\n\nclass AgentToolCall(BaseModel):\n    """Trace of a tool invocation the agent performed."""\n\n    tool: str\n    arguments: dict[str, Any]\n    output_preview: str\n\n\nclass AgentRecommendation(BaseModel):\n    """Action item recommended by the agent."""\n\n    title: str\n    detail: str\n\n\nclass AgentRunContext(BaseModel):\n    """Input payload submitted by the frontend."""\n\n    feature_slug: str = Field(..., min_length=2, max_length=40)\n    launch_date: date\n    audience_role: str = Field(..., min_length=2, max_length=60)\n    audience_experience: Literal["beginner", "intermediate", "advanced"]\n    include_risks: bool = Field(default=True)\n\n\nclass AgentRunResult(BaseModel):\n    """Structured result returned to the frontend."""\n\n    summary: str\n    recommended_actions: list[AgentRecommendation]\n    plan: Plan\n    tool_calls: list[AgentToolCall]\n\n\ndef run_release_readiness_agent(context: AgentRunContext) -> AgentRunResult:\n    """Coordinate tool calls to prepare a release readiness brief."""\n\n    try:\n        brief: FeatureBrief = fetch_feature_brief(context.feature_slug)\n    except KeyError as exc:\n        raise AgentServiceError(f"Unknown feature \'{context.feature_slug}\'.") from exc\n\n    try:\n        launch_window: LaunchWindow = fetch_launch_window(context.feature_slug)\n    except KeyError as exc:\n        raise AgentServiceError(\n            f"Launch window data is missing for feature \'{context.feature_slug}\'."\n        ) from exc\n\n    contacts: list[SupportContact] = fetch_support_contacts(context.audience_role)\n    slo_watch_items: list[str] = list_slo_watch_items(context.feature_slug)\n\n    tool_calls = [\n        AgentToolCall(\n            tool="fetch_feature_brief",\n            arguments={"feature_slug": context.feature_slug},\n            output_preview=f"{brief.name}: {brief.summary}",\n        ),\n        AgentToolCall(\n            tool="fetch_launch_window",\n            arguments={"feature_slug": context.feature_slug},\n            output_preview=(\n                f"{launch_window.environment} window {launch_window.window_start.isoformat()}"\n                f" → {launch_window.window_end.isoformat()}"\n            ),\n        ),\n        AgentToolCall(\n            tool="fetch_support_contacts",\n            arguments={"audience_role": context.audience_role},\n            output_preview=f"{len(contacts)} contact(s) notified",\n        ),\n    ]\n\n    if slo_watch_items:\n        tool_calls.append(\n            AgentToolCall(\n                tool="list_slo_watch_items",\n                arguments={"feature_slug": context.feature_slug},\n                output_preview=", ".join(slo_watch_items[:2]),\n            )\n        )\n\n    plan_request = PlanRequest(\n        goal=f"Launch {brief.name} successfully",\n        audience_role=context.audience_role,\n        audience_experience=context.audience_experience,  # type: ignore[arg-type]\n        primary_risk=slo_watch_items[0] if context.include_risks and slo_watch_items else None,\n    )\n    plan: Plan = build_plan(plan_request)\n\n    summary = (\n        f"{brief.name} targets {brief.audience_role} personas. "\n        f"Production window: {launch_window.window_start:%b %d}–{launch_window.window_end:%b %d}. "\n        f"Success metric: {brief.success_metric}."\n    )\n\n    recommended_actions: list[AgentRecommendation] = [\n        AgentRecommendation(\n            title="Confirm launch communications",\n            detail=(\n                f"Share the feature brief with {contacts[0].contact} and align on messaging for the "\n                f"{launch_window.environment} window."\n            ),\n        ),\n        AgentRecommendation(\n            title="Validate operational readiness",\n            detail=(\n                "Ensure runbooks and dashboards reflect the new flow. Coordinate with site reliability "\n                "for rollout approval."\n            ),\n        ),\n    ]\n\n    if context.include_risks and slo_watch_items:\n        recommended_actions.append(\n            AgentRecommendation(\n                title="Mitigate top risk",\n                detail=f"Create a mitigation plan for: {slo_watch_items[0]}.",\n            )\n        )\n\n    if len(contacts) > 1:\n        recommended_actions.append(\n            AgentRecommendation(\n                title="Broadcast stakeholder update",\n                detail=(\n                    "Send a tailored update to secondary contacts so downstream teams can prepare "\n                    "training materials and support docs."\n                ),\n            )\n        )\n\n    return AgentRunResult(\n        summary=summary,\n        recommended_actions=recommended_actions,\n        plan=plan,\n        tool_calls=tool_calls,\n    )\n')
print('Agent service module updated.')

### Step 3: API router
Expose the agent through FastAPI using `app/routers/agent.py` so the frontend can call it.

In [ ]:
from pathlib import Path

agent_router_path = Path('ai-web/backend/app/routers/agent.py')
agent_router_path.parent.mkdir(parents=True, exist_ok=True)
agent_router_path.write_text('"""Agent endpoints that power the Lab 05 release readiness workflow."""\n\nfrom __future__ import annotations\n\nfrom fastapi import APIRouter, HTTPException\n\nfrom app.services.agent import (\n    AgentRunContext,\n    AgentRunResult,\n    AgentServiceError,\n    run_release_readiness_agent,\n)\n\nrouter = APIRouter(prefix="/ai", tags=["ai"])\n\n\n@router.post("/release-readiness", response_model=AgentRunResult)\ndef release_readiness(payload: AgentRunContext) -> AgentRunResult:\n    """Run the deterministic agent pipeline and surface structured output."""\n\n    try:\n        return run_release_readiness_agent(payload)\n    except AgentServiceError as exc:\n        raise HTTPException(status_code=404, detail=str(exc)) from exc\n')
print('Agent router available.')

### Step 4: Confirm router registration
Ensure `app/main.py` includes the new agent router alongside existing imports.

In [ ]:
from pathlib import Path

main_path = Path('ai-web/backend/app/main.py')
text = main_path.read_text()
if 'agent_router' not in text:
    text = text.replace("from app.routers.echo import router as echo_router\nfrom app.routers.gemini import router as gemini_router\n",
        "from app.routers.agent import router as agent_router\nfrom app.routers.echo import router as echo_router\nfrom app.routers.gemini import router as gemini_router\nfrom app.routers.planner import router as planner_router\n")
    text = text.replace("app.include_router(echo_router)\napp.include_router(gemini_router)\n",
        "app.include_router(agent_router)\napp.include_router(echo_router)\napp.include_router(gemini_router)\napp.include_router(planner_router)\n")
    main_path.write_text(text)
    print('Agent router registered in FastAPI app.')
else:
    print('Agent router already configured.')

### Step 5: Execute the agent locally
Call the service helper directly to preview the structured JSON returned to the frontend.

In [ ]:
from datetime import date

from app.services.agent import AgentRunContext, run_release_readiness_agent

context = AgentRunContext(
    feature_slug='curriculum-pathways',
    launch_date=date(2025, 3, 10),
    audience_role='Instructor',
    audience_experience='intermediate',
)

result = run_release_readiness_agent(context)
result.model_dump()

## Validation / acceptance checks
```bash
# locally
curl -X POST http://localhost:8000/ai/release-readiness \
  -H 'Content-Type: application/json' \
  -d '{"feature_slug":"curriculum-pathways","launch_date":"2025-03-10","audience_role":"Instructor","audience_experience":"intermediate"}'
```
- The response includes a summary, recommended actions, tool call traces, and a nested planner payload.
- The FastAPI interactive docs (`/docs`) display the new endpoint under the **ai** tag.
- React development mode renders the structured agent output without runtime warnings.

## Homework / extensions
- Add more feature briefs and launch windows to the tool module, then branch the agent logic based on feature type.
- Persist agent responses or tool call traces to an analytics datastore for auditing.
- Connect the agent output to the frontend planner UI so students can compare generated steps against the release readiness summary.


## Updated full-stack template walkthrough

This lab now uses a Postgres-backed stack managed through Alembic migrations and Docker Compose.

- **Database + Alembic**: `backend/alembic/env.py` and `backend/alembic/versions/20250212_initial.py` create tables for echo retries, planner runs, course resources, and RAG document chunks. Run migrations with `alembic upgrade head` (the backend container executes this automatically on start).
- **FastAPI wiring**: `backend/app/database.py` exposes a SQLAlchemy `SessionLocal` dependency. Routers like `app/routers/echo.py`, `app/routers/planner.py`, and `app/routers/resources.py` read and write real rows instead of in-memory mocks.
- **Docker Compose + Nginx**: `docker-compose.yml` now launches Postgres, FastAPI (with migrations), the Vite dev server, and an Nginx reverse proxy (`nginx/default.conf`) that fronts both the API (`/api`) and frontend assets.
- **RAG chatbot**: `app/services/rag.py` indexes seeded `DocumentChunk` rows, while `app/services/chatbot.py` blends retrieval with Gemini when `GEMINI_API_KEY` is configured. The `/chat` route exposes the agent-like flow and the React `ChatPanel` renders responses and retrieved context.
- **New end-to-end feature**: The `resources` table powers the Resource Board UI (`frontend/src/features/resources`) so students can add persistent links. Planner history (`frontend/src/features/planner`) and echo retries now read from Postgres as well.

### How to run the stack with Docker Compose

1. `cd ai-web`
2. `docker compose up --build`
3. Open http://localhost:8080 to reach Nginx. API requests are proxied to FastAPI at `/api`. Postgres data lives in the `db_data` volume.

### Creating and applying new migrations

1. Enter the backend container: `docker compose exec backend bash`
2. Generate a migration: `alembic revision -m "describe change" --autogenerate`
3. Apply migrations: `alembic upgrade head`

### Testing the new features

- **Echo retry + persistence**: Submit the echo form; the "Recent echo attempts" list should update from the `echo_attempts` table.
- **Resource Board**: Add a URL in the Resource Board. Refreshing the page keeps entries thanks to the `resources` table.
- **Planner + history**: Generate a plan in the Planner panel; the newest plan appears in the history list powered by the `plan_runs` table.
- **Chatbot/agent UI**: Ask deployment or migration questions in the chatbot. Retrieved context from `document_chunks` is displayed alongside agent steps; Gemini responses are used when `GEMINI_API_KEY` is provided.

Refer to the updated source files when walking through the lab so students can trace how migrations, database sessions, and the React UI connect end to end.
